In [69]:
import os
import json
import random
import sys

Loen sisse käsitsi märgendatud protokollid, mida ei tohiks ühes alamosas olla:

In [70]:
def read_protocols_tagged_by_hand(file = "k2sitsi_morfiga_protokollid.txt"):
    protocols_tagged_by_hand = []
    with open(file, 'r', encoding="UTF-8") as f:
        lines = f.readlines()
    for line in lines:
        line = line.replace("\n", "")
        if line != "" and (line != "k2sitsi_morfiga_protokollid2:" and line != "k2sitsi_morfiga_protokollid1:"):
            line += ".json"
            protocols_tagged_by_hand.append(line)
            
    return protocols_tagged_by_hand

Ideaalne jaotus ehk terve korpuse jaotus:

In [71]:
def calculate_whole_corpus_distribution(filenames = os.listdir("vallakohtufailid_json/")):
    all_annotations = []
    for file in filenames:
        with open("vallakohtufailid_json/" + file, 'r', encoding="UTF-8)") as f:
            data = json.load(f)
            for dictionary in data.get('layers')[0].get('spans'):
                all_annotations.append(dictionary.get('annotations')[0].get('nertag'))
    per = all_annotations.count("PER") / len(all_annotations) * 100
    loc_org = all_annotations.count("LOC_ORG") / len(all_annotations) * 100
    loc = all_annotations.count("LOC") / len(all_annotations) * 100
    org = all_annotations.count("ORG") / len(all_annotations) * 100
    misc = all_annotations.count("MISC") / len(all_annotations) * 100

    ideal_distribution = {'PER': per, 'LOC_ORG' : loc_org, 'LOC' : loc, 'ORG' : org, 'MISC' : misc}
    
    return ideal_distribution

Kõikide dokumentide NE statistika:

In [72]:
def find_document_ne_statistics(filenames = os.listdir("vallakohtufailid_json/")):
    statistics = dict()
    for file in filenames:
        ner_annotations = []
        with open("vallakohtufailid_json/" + file, 'r', encoding="UTF-8)") as f:
            data = json.load(f)
            for dictionary in data.get('layers')[0].get('spans'):
                ner_annotations.append(dictionary.get('annotations')[0].get('nertag'))

        statistics_for_file = dict()
        for annotation in set(ner_annotations):
            statistics_for_file[str(annotation)] = ner_annotations.count(annotation)
        statistics[file] = statistics_for_file

    return statistics

Taaskasutatavad muutujad:

In [73]:
statistics = find_document_ne_statistics()
ideal_distribution = calculate_whole_corpus_distribution()
protocols_tagged_by_hand = read_protocols_tagged_by_hand()

In [74]:
def swap_items_in_lists(A, B, i, j):
    temp_b = B[j]
    B[j] = A[i]
    A[i] = temp_b
    return A, B

def improve_scores(largest, second_largest):
    score_largest_old = calculate_score(largest)
    score_second_largest_old = calculate_score(second_largest)
    for i in range(len(largest)):
        for j in range(len(second_largest)):
            largest, second_largest = swap_items_in_lists(largest, second_largest, i, j)
            
            score_largest_new = calculate_score(largest)
            score_second_largest_new = calculate_score(second_largest)

            if score_largest_old > score_largest_new and score_second_largest_old > score_second_largest_new:
                print(f"{i, j} Skoor paranes mõlema alamkorpuse lõikes.")
                score_largest_old = score_largest_new
                score_second_largest_old = score_second_largest_new
                i += 1
            elif score_largest_old < score_largest_new and score_second_largest_old > score_second_largest_new:
                print(f"{i, j} Skoor paranes ainult esimeses massiivis.")
                swap_items_in_lists(largest, second_largest, i, j)
                score_largest_old = score_largest_old
                score_second_largest_old = score_second_largest_old
            elif score_largest_old > score_largest_new and score_second_largest_old < score_second_largest_new:
                print(f"{i, j} Skoor paranes ainult teises massiivis.")
                swap_items_in_lists(largest, second_largest, i, j)
                score_largest_old = score_largest_old
                score_second_largest_old = score_second_largest_old
            else:
                print(f"{i, j} Skoor ei paranenud kummaski massiivis.")
                swap_items_in_lists(largest, second_largest, i, j)
                score_largest_old = score_largest_old
                score_second_largest_old = score_second_largest_old
    return largest, second_largest

In [93]:
def get_rid_of_hand_tagged(least_hand_tagged, other_files):
    for i, x in enumerate(set(least_hand_tagged).intersection(protocols_tagged_by_hand)):
        print(i+1, x)
        index = least_hand_tagged.index(x)
        score_old = calculate_score(least_hand_tagged)

        for files in other_files:
            breaking = "no"
            score_old_output = calculate_score(files)

            for j, y in enumerate(files):
                if y in protocols_tagged_by_hand:
                    continue
                else:
                    swap_items_in_lists(least_hand_tagged, files, index, j)
                    score_new = calculate_score(least_hand_tagged)
                    score_new_output = calculate_score(files)
                    if score_new < score_old and score_new_output < score_old_output:
                        print("Leidsin vaste! Vahetasin failiga", y)
                        breaking = "yes"
                        break
                    elif score_new == score_old and score_new_output == score_old_output or \
                         abs(score_new - score_old) < 0.05 and abs(score_new_output-score_old_output) < 0.05:
                        print("Leidsin vaste! Vahetasin failiga", y)
                        breaking = "yes"
                        break
                    else:
                        swap_items_in_lists(least_hand_tagged, files, index, j)
                        continue
            if breaking == "yes":
                break
    return least_hand_tagged, other_files

Funktsioon iga alajaotuse proportsioonide leidmiseks (protsentuaalne):

In [76]:
def calculate_proportions(filenames):
    statistics_for_proportion = {}
    
    for file in filenames:
        statistics_for_proportion[file] = statistics[file]

    all_annotations = list()
    for item in statistics_for_proportion.values():
        for key in item:
            for i in range(item[key] + 1):
                all_annotations.append(key)
                
    proportions = dict()
    for file in statistics_for_proportion:
        for key in statistics_for_proportion[file].keys():
            proportion = all_annotations.count(key) / len(all_annotations) * 100
            proportions[key] = proportion
        
    return proportions

Juhujaotuse genereerimine kõikidest failidest ning selle jagamine n-ks võrdseks osaks:

In [77]:
def n_even_chunks(filenames, n):
    files = []
    last = 0
    for i in range(1, n+1):
        current = int(round(i* (len(filenames) / n)))
        files.append(filenames[last:current])
        last = current
    return files

def generate_random_division(filenames, n):
    random_distributions = []
    for i in range(n):
        filenames = random.sample(filenames, len(filenames))
        random_distributions.append(filenames)
    
    return random_distributions

Skooride kalkuleerimine võrreldes ideaalse jaotusega:

In [78]:
def calculate_score(filenames):
    proportions = calculate_proportions(filenames)
    score = 0
    for proportion in proportions:
        ideal_distribution_proportion = ideal_distribution[proportion]
        current_proportion = proportions[proportion]
        
        if current_proportion == ideal_distribution_proportion:
            score += 0
        else:
            score += abs(ideal_distribution_proportion - current_proportion)
    
    return score

In [94]:
seed = 1
random.seed(seed)
print("Seed:", seed)

subdistributions = list()
random_distributions = generate_random_division(sorted(os.listdir("vallakohtufailid_json/")), 25)
for distribution in random_distributions:
    distribution = n_even_chunks(distribution, 6)
    subdistributions.append(distribution)
    
sums_of_distributions = list()
list_of_scores = list()

for distribution in subdistributions:
    scores = dict()
    for files in distribution:
        score = calculate_score(files)
        scores[score] = files
        print(score)
    list_of_scores.append(scores)
    print(f"Score of sub-corpus: {sum(scores.keys()) / len(scores.keys())}")
    print(" ")

for listofscores in list_of_scores:    
    sums_of_distributions.append(sum(listofscores.keys()) / len(listofscores.keys()))

Seed: 1
6.207355815198964
11.874719943138807
11.39658420239689
9.17197059987578
7.268100311057957
5.7046876800153585
Score of sub-corpus: 8.603903091947293
 
9.9732211280256
9.456529460492243
6.053995218755706
9.769595602711254
8.059186637618012
9.10155524753753
Score of sub-corpus: 8.735680549190057
 
8.91464336784733
11.537179912708115
10.662136280342123
8.995510057232245
7.9614464877588595
4.567379134442525
Score of sub-corpus: 8.773049206721867
 
6.610682850010091
4.410398036932767
14.603704211547347
9.359958462105022
9.36324726896274
8.198438151955253
Score of sub-corpus: 8.757738163585536
 
9.002350320924412
8.496119409176766
4.983825179903615
9.818771068405251
12.538850800304711
7.734536626649881
Score of sub-corpus: 8.762408900894107
 
5.625934661383683
8.289162864722709
4.899051352860271
12.658019940409547
9.364925895845962
11.674180489944426
Score of sub-corpus: 8.751879200861099
 
11.088417032091648
10.51824569911324
8.806274374536615
6.7418000338537
6.471232066336358
8.2096

In [80]:
'''
smallest_score = list_of_scores[sums_of_distributions.index(min(sums_of_distributions))]


while (sum(smallest_score.keys()) / len(smallest_score.keys())) >= 5:
    keys_list = sorted(smallest_score.keys())
    largest_key = keys_list[-1]
    second_largest_key = keys_list[-2]
    
    largest = smallest_score[largest_key]
    second_largest = smallest_score[second_largest_key]
    
    print(f"Algsed skoorid on {keys_list[-1]} ning {keys_list[-2]}.")
    largest, second_largest = improve_scores(largest, second_largest)
    score_after_improving_largest = calculate_score(largest)
    score_after_improving_second_largest = calculate_score(second_largest)
    print(f"Skoorid pärast parandamist on {score_after_improving_largest} ning {score_after_improving_second_largest}.")
    del smallest_score[largest_key]
    del smallest_score[second_largest_key]
    smallest_score[score_after_improving_largest] = largest
    smallest_score[score_after_improving_second_largest] = second_largest
'''

'\nsmallest_score = list_of_scores[sums_of_distributions.index(min(sums_of_distributions))]\n\n\nwhile (sum(smallest_score.keys()) / len(smallest_score.keys())) >= 5:\n    keys_list = sorted(smallest_score.keys())\n    largest_key = keys_list[-1]\n    second_largest_key = keys_list[-2]\n    \n    largest = smallest_score[largest_key]\n    second_largest = smallest_score[second_largest_key]\n    \n    print(f"Algsed skoorid on {keys_list[-1]} ning {keys_list[-2]}.")\n    largest, second_largest = improve_scores(largest, second_largest)\n    score_after_improving_largest = calculate_score(largest)\n    score_after_improving_second_largest = calculate_score(second_largest)\n    print(f"Skoorid pärast parandamist on {score_after_improving_largest} ning {score_after_improving_second_largest}.")\n    del smallest_score[largest_key]\n    del smallest_score[second_largest_key]\n    smallest_score[score_after_improving_largest] = largest\n    smallest_score[score_after_improving_second_largest]

In [95]:
smallest_score = list_of_scores[sums_of_distributions.index(min(sums_of_distributions))]

number_of_hand_tagged_protocols = []
files_to_use = []
final_files = []
other_files = []

for files in smallest_score.values():
    number_of_hand_tagged_protocols.append(len(set(files).intersection(protocols_tagged_by_hand)))
    files_to_use.append(files)

least_hand_tagged_old = files_to_use[number_of_hand_tagged_protocols.index(min(number_of_hand_tagged_protocols))]

for files in files_to_use:
    other_files.append(files)
other_files.pop(other_files.index(least_hand_tagged_old))

least_hand_tagged, other_files = get_rid_of_hand_tagged(least_hand_tagged_old, other_files)

for files in other_files:
    final_files.append(files)

1 V6ru_R2pina_Kahkva_id24674_1868a.json
Leidsin vaste! Vahetasin failiga Tartu_R6ngu_Aakre_id12415_1827a.json
2 L22ne_Martna_Martna_id14205_1869a.json
Leidsin vaste! Vahetasin failiga Viru_Haljala_Vihula_id8056_1877a.json
3 J2rva_Peetri_V2ike-Kareda_id22448_1881a.json
Leidsin vaste! Vahetasin failiga Saare_Kihelkonna_Atla_id7551_1875a.json


In [96]:
file = open("divided_corpus.txt", "w")
iterator = 1
for files in final_files:
    for f in files:
        file.write(str(f) + ":" + str(iterator) + "\n")
    iterator += 1
file.close()

file = open("corpus_subdistribution_without_hand_tagged.txt", "w")
for f in least_hand_tagged:
    file.write(str(f) + ":" + str(iterator) + "\n")
file.close()

In [100]:
for files in final_files:
    print("Alajaotuses olevad käsitsi märgendatud failid:", len(set(files).intersection(protocols_tagged_by_hand)))
    print("Alajaotuse skoor:", str(calculate_score(files)) + "\n")
print("Alajaotuses olevad käsitsi märgendatud failid:", len(set(least_hand_tagged).intersection(protocols_tagged_by_hand)))
print("Alajaotuse skoor:", calculate_score(least_hand_tagged))

Alajaotuses olevad käsitsi märgendatud failid: 13
Alajaotuse skoor: 6.047574351069665

Alajaotuses olevad käsitsi märgendatud failid: 13
Alajaotuse skoor: 7.944807552650694

Alajaotuses olevad käsitsi märgendatud failid: 8
Alajaotuse skoor: 9.63458779587663

Alajaotuses olevad käsitsi märgendatud failid: 10
Alajaotuse skoor: 9.392619000462133

Alajaotuses olevad käsitsi märgendatud failid: 12
Alajaotuse skoor: 9.92939691555241

Alajaotuses olevad käsitsi märgendatud failid: 0
Alajaotuse skoor: 8.203464206801717
